## Нейросети - сверточные сети.
### Ноутбук составлен по воспоминаниям из разных курсов, в первую очередь Stanford cs231n, Jeremy Howard Fast AI, а также по материалам сайта https://www.jefkine.com/general/2016/09/05/backpropagation-in-convolutional-neural-networks/

In [1]:
import numpy as np

In [2]:
def pad_with_zeros(X):
    X_pad = np.zeros((X.shape[0] + 2, X.shape[1] + 2))
    X_pad[1:-1,1:-1] = X
    return X_pad

In [3]:
X = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]])

In [4]:
K = np.array([[0.0, 1.0], [2.0, 3.0]])

In [5]:
def conv2d(X, K, pad = True):
    if pad:
        X = pad_with_zeros(X)
        
    h, w = X.shape[0], X.shape[1]
    k1, k2 = K.shape[0], K.shape[1]
    result = np.zeros((h - k1 + 1, w - k2 + 1))
    
    for i in range(0 + k1 - 1, result.shape[0] + k1 - 1):
        for j in range(0 + k2 - 1, result.shape[1] + k2 - 1):
            result[i - k1 + 1, j - k2 + 1] = (X[i - k1 + 1: i + 1, j - k2 + 1: j + 1] * K).sum()
    return result

In [6]:
conv2d(X, K)

array([[ 3.,  8., 13.,  6.],
       [13., 25., 31., 12.],
       [25., 43., 49., 18.],
       [ 7.,  8.,  9.,  0.]])

Попробуем сначала простой слой с одним фильтром - 1 канал на входе, 1 на выходе

In [7]:
def rot180(X):
    return np.rot90(np.rot90(X))

In [19]:
class ConvLayer:
    
    def __init__(self, kernel_size = 3, pad = False):
        self.kernel = np.random.default_rng().uniform(-1, 1, (kernel_size, kernel_size)).astype(np.float32)
        self.pad = pad
        self.X_ = None
        
    def forward(self, X):
        self.X_ = X
        return conv2d(X, self.kernel, pad = self.pad)
    
    def backward(self, upstr_grad, lr = 0.01, debug = 0):
        grad_w = conv2d(self.X_, rot180(upstr_grad), pad=self.pad)
        print(grad_w)
                

In [20]:
CL = ConvLayer(kernel_size = 2)

In [21]:
CL.forward(X)

array([[3.93207132, 5.44062284],
       [8.4577259 , 9.96627742]])

In [22]:
upstr_grad = np.array([[0, 1], [2, 1]])

In [23]:
CL.backward(upstr_grad)

[[ 9. 13.]
 [21. 25.]]


Без padding ответ верный, попробуем с padding

In [24]:
CL = ConvLayer(kernel_size = 3, pad = True)

In [25]:
CL.forward(X)

array([[ -0.55507277,  -3.25769332,  -8.646089  ],
       [ -0.1937833 ,  -7.34973274, -14.95768483],
       [  5.63369469,  -1.06019308,  -5.49231242]])

In [26]:
upstr_grad = np.array([[0, 1, 2], [1, 2, 3], [2, 3, 0]])

In [27]:
CL.backward(upstr_grad)

[[ 8. 23. 28.]
 [27. 62. 61.]
 [44. 73. 60.]]


С padding тоже верно